In [120]:
import pandas as pd
import re

#csv에서 news content를 txt로 변환

csv=pd.read_csv('../Data/201905.csv',encoding='utf-8-sig')

text=csv['text']

with open('News_Contents.txt','w',encoding='utf-8-sig') as file:
    for item in text:
        test = re.sub(r'\([^)]*\)|\[[^)]*\]|[/@]', '', item)
        file.write("%s\n" % test)

In [121]:
import pandas as pd

df = pd.read_csv('../Data/201905.csv', encoding='utf-8-sig')
df.tail()

,date,press,category,headline,url,text,img
22832,2019/05/31,서울신문,사회,‘집단성폭행’ 최종훈 “구속여부 다시 판단을”…법원 기각,http://www.seoul.co.kr/news/newsView.php?id=20...,성폭력처벌법상 특수준강간 등의 혐의를 받고 있는 가수 최종훈이 9일 서울 서초구 서...,['//img.seoul.co.kr/img/upload/2019/05/09/SSI_...
22833,2019/05/31,세계일보,사회,“또 개물림 사고” 목줄 풀린 ‘말라뮤트’가 초등생 물어 “머리 등 2∼3㎝가량 찢어져”,http://www.segye.com/content/html/2019/05/31/2...,경기 수원시 장안구 한 놀이터에서 대형견인 말라뮤트가 놀고 있는 초등생을 무는 사고...,['//img.segye.com/content/image/2019/05/31/201...
22834,2019/05/31,동아일보,국제,부다페스트 유람선 사고 유발한 크루즈선 소유주는 노르웨이 출신 사업가,http://news.donga.com/3/all/20190531/95795499/1,29일(현지 시간)헝가리 부다페스트 다뉴브강에서 한국인 관광객 등 35명을 태운 유...,['https://dimg.donga.com/wps/NEWS/IMAGE/2019/0...
22835,2019/05/31,중앙일보,"사회,지역","올해 대학진학률 76.5%…강남구 46.8%, 가장 낮은 이유는?",https://news.joins.com/article/olink/23077732,서울 내에서 강남구 대학진학률이 46.8%로 25개 자치구 중 최저인 것으로 나타났...,['https://pds.joins.com/news/component/htmlpho...
22836,2019/05/31,세계일보,국제,"헝가리 외교장관 ""선체인양 온힘 다할 것""",http://www.segye.com/content/html/2019/05/31/2...,"강경화 장관과 공동회견""강 하류 인접 국가들에도 수색협조 요청""세월호 참사, 자국의...",['//img.segye.com/content/image/2019/05/31/201...


In [122]:
from kiwipiepy import Kiwi
import pandas as pd
import numpy as np
import math
import tomotopy as tp

kiwi = Kiwi()
kiwi.load_user_dictionary('sorted_wordlist_end.txt')
kiwi.prepare()

stopword = []
def rm_stopword(stopword_file): 
    files = open(stopword_file, 'r', encoding="utf-8")
    while True: #불용어 리스트 생성
        line = files.readline()
        if not line: break
        wordlist = line.split('\t')
        if wordlist[1].startswith('N'):
            stopword.append(wordlist[0])
    stopwords = set(stopword)
    files.close()
    return stopword

def npmi(a, b, n, z):
    return (math.log((n*z)/(a*b))) / (math.log(n/z))

def extract_topic(doc):
    model = tp.LDAModel(k=1, min_cf=2, min_df=1, tw=tp.TermWeight.PMI)
    for line in doc:
        model.add_doc(line) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = []
    for j in range(0,100):
        model.train(10) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        for w, p in res:
            s.append(w)
        # s = ', '.join(w for w, p in res)
    return ', '.join(s)

In [123]:
from kss import split_sentences
from collections import Counter
import pprint as pp
import re

rm_stopword('한국어불용어100.txt')
docs = []
topics = []
line_topics = []
for news in open('News_Contents.txt', encoding='utf-8'):
    doc = []
    com = []
    lines = split_sentences(news)
    context = []
    #형태소 분석
    for res in kiwi.analyze(lines):
        content = [word.strip() for word, tag, _, _ in res[0][0] if tag.startswith('N') and word not in stopword] #불용어사전 적용
        doc.append(content)
    docs.append(doc)
    line_topics.append(extract_topic(doc))

    if len(docs) % 1000 == 0:
        print(len(docs))
df['N_list'] = docs
df['Topics'] = line_topics
df.to_csv('../Topic_Abstract/201905_LDA.csv', encoding='utf-8-sig', index=False)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
